In [1]:
import numpy as np
import math
import random
import itertools
import sys
import pandas as pd
from sklearn.datasets import load_iris
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
import json

In [2]:
def normalize_vector(vector):
    '''
    Returns the normalized vector of the provided one, a length 1
    vector with the same direction as the original.
    '''
    vector = np.array(vector)
    return (vector/math.sqrt(np.sum(pow(vector, 2)))).tolist()

In [3]:
def extended_normalization(vector):
    '''
    When two vectors of different lengths but in the same direction are normalized, 
    the result will be the same for both of them.
    
    In order to avoid collisions of normalized vectors, an extended normalization can
    be applied, where an extra dimension is added to the vector to be normalized. By
    adding a component of length 1 in the new dimension and normalizing the result,
    it is ensured that no two vectors which were originally in the same dimensions
    will produce the same output.
    '''
    vector = np.append(vector, 1)
    return (vector/math.sqrt(np.sum(pow(vector, 2)))).tolist()

In [4]:
def get_random_unit_vector(components):
    '''
    Returns a random vector of length 1 with the required number of components
    '''
    #return normalize_vector([random.random() for i in range(components)])    
    return normalize_vector([random.uniform(-1, 1) for i in range(components)])

In [5]:
class SOM:
    
    def __init__ (self, dimensions, topology='rectangular', max_iter=100, 
                  initial_learning_rate=0.1, initial_neighborhood_radius=0,
                 prediction_mode="neuron-labelling", labels=[], output_layer=[]):
        self.dimensions = dimensions
        self.output_layer = None if output_layer==[] else output_layer
        self.max_iter = max_iter
        self.topology = topology
        self.initial_learning_rate = initial_learning_rate
        self.initial_neighborhood_radius = initial_neighborhood_radius if\
                                    initial_neighborhood_radius>0 else\
                                    math.floor((np.min(dimensions)-1)/2)
        self.prediction_mode = prediction_mode
        indexes_list = [list(range(i)) for i in self.dimensions]
        self.output_layer_indexes = list(itertools.product(*indexes_list))
        if self.prediction_mode=="mode":
            self.neuron_class_count = np.zeros(self.dimensions, dtype=object)
        if labels == []:
            self.labels = np.zeros(self.dimensions, dtype=object)
            for i in self.output_layer_indexes:
                self.labels[i]=[None, math.inf]
        else:
            self.labels = labels
        return
    
    def __weight_initialization(self, num_attr):
        self.output_layer = np.zeros(self.dimensions, dtype=object)
        for i in self.output_layer_indexes:
            self.output_layer[i] = get_random_unit_vector(num_attr)
        return
    
    def __get_output(self, instance):
        output_matrix = np.zeros(self.dimensions)
        for i in self.output_layer_indexes:
            output_matrix[i] = distance.euclidean(instance, self.output_layer[i])
        return output_matrix

    def __update_labels(self, output_matrix, instance_class):
        for i in self.output_layer_indexes:
            if output_matrix[i] < self.labels[i][1]:
                self.labels[i][0]=int(instance_class)
                self.labels[i][1]=float(output_matrix[i])
        return
    
    def __get_neighbors(self, index, radius):
        indexes_list = [list(range(i-math.ceil(radius/2), i+math.ceil(radius/2)+1)) for i in index]
        indexes = np.array(list(itertools.product(*indexes_list)))
        return [tuple(i) for i in indexes%self.dimensions]
    
    def fit(self, X, Y):
        self.__weight_initialization(len(X[0]) + 1)
        X = [extended_normalization(v) for v in X]
        
        iteration=1
        for epoch in range(self.max_iter):
            neighborhood_radius = max(self.initial_neighborhood_radius-epoch, 0)
            instance_indexes=list(range(len(X)))
            random.shuffle(instance_indexes)
            
            for instance_index in instance_indexes:
                learning_rate = self.initial_learning_rate/(1+(iteration/len(X)))
                iteration+=1
                input_vector = X[instance_index]
                output_matrix = self.__get_output(input_vector)
                best_matching_unit = np.unravel_index(np.argmin(output_matrix, axis=None), 
                                                      output_matrix.shape)

                neighbors = self.__get_neighbors(best_matching_unit, neighborhood_radius)

                for n in neighbors:
                    self.output_layer[n] = normalize_vector(self.output_layer[n]+
                                                            learning_rate*np.array(input_vector))
        
        if self.prediction_mode=="mode":
            classes = list(np.unique(Y))
            for i in range(len(classes)):
                if type(classes[i])==np.int64:
                    classes[i] = int(classes[i])
            for i in self.output_layer_indexes:
                self.neuron_class_count[i]={}
                for c in classes:
                    self.neuron_class_count[i][c]=0
            for instance_index in range(len(X)):
                output_matrix = self.__get_output(X[instance_index])
                best_matching_unit=np.unravel_index(np.argmin(output_matrix, axis=None), 
                                                              output_matrix.shape)
                self.neuron_class_count[best_matching_unit][Y[instance_index]]+=1
            for i in self.output_layer_indexes:
                self.labels[i][0] = classes[np.argmax([self.neuron_class_count[i][k]\
                                                    for k in self.neuron_class_count[i]])]
                self.labels[i][1] = 0
                
        elif self.prediction_mode=="neuron-labelling":
            for instance_index in range(len(X)):
                output_matrix = self.__get_output(X[instance_index])
                self.__update_labels(output_matrix, Y[instance_index])
        return
    def predict(self, X):
        X = [extended_normalization(v) for v in X]
        predictions = []
        for instance in X:
            output_matrix = self.__get_output(instance)
            best_matching_unit = np.unravel_index(np.argmin(output_matrix, axis=None), 
                                                  output_matrix.shape)
            predictions.append(self.labels[best_matching_unit][0])
        return predictions

In [6]:
mnist = fetch_openml('mnist_784',version=1)

In [7]:
mnist_ext = np.append(mnist.data, np.ones((mnist.data.shape[0],1),dtype=float).reshape(-1,1),axis=1)

In [8]:
X=normalize(mnist.data)
Y=[]
for i in range(mnist.target.shape[0]):
    Y.append(int(mnist.target[i]))
X_train, X_test, y_train, y_test = train_test_split(X[:1000], Y[:1000], test_size=0.33, random_state=0)

In [9]:
som = SOM([15, 9], max_iter=20, initial_learning_rate=0.1, initial_neighborhood_radius=4)
som.fit(X_train, y_train)
prediction = som.predict(X_test)
accuracy_score(y_test, prediction, normalize=True)

0.7363636363636363